# Derivation of a Quasi-Quintic Derivation of Divergence
## Plan
- Find a finite difference approximation for $\partial\Psi/\partial x$
- Express this as a differences of fluxes
- Express using local gradients

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
from sympy import latex
from sympy.matrices import Matrix, MatrixSymbol
from fractions import Fraction as Fr
from sympy import I, E, pi, Q

## Quintic finite difference approximation for $\partial\Psi/\partial x$
Assume that $\Psi$ follows a quintic polynomial:
\begin{eqnarray}
\psi &=& ax^5 + bx^4 + cx^3 + dx^2 + ex + f
\end{eqnarray}
with $\Delta x=1$ and $x=0$ at $j=0$. We know solutions
\begin{eqnarray}
\psi_{j-3} = \psi_0 &=& (-3)^5 a + (-3)^4 b + (-3)^3 c + (-3)^2 d - 3e + f \\
\psi_{j-2} = \psi_1 &=& (-2)^5 a + (-2)^4 b + (-2)^3 c + (-2)^2 d - 2e + f \\
\psi_{j-1} = \psi_2 &=& -a + b - c + d - e + f \\
\psi_{j}   = \psi_3 &=& f \\
\psi_{j+1} = \psi_4 &=& a + b + c + d + e + f \\
\psi_{j+2} = \psi_5 &=& 2^5 a + 2^4 b + 2^3 c + 2^2 d + 2e + f
\end{eqnarray}
So $(a,b,c,d,e,f)$ is the solution of the matrix equation
\begin{equation*}
\left( \begin{array}{cccccc}
-3^5 & 3^4 & -3^3 & 3^2 & -3 & 1 \\
-2^5 & 2^4 & -2^3 & 2^2 & -2 & 1 \\
-1   & 1 & -1 & 1 & -1 & 1 \\
0 & 0 & 0 & 0 & 0 & 1 \\
1 & 1 & 1 & 1 & 1 & 1 \\
2^5 & 2^4 & 2^3 & 2^2 & 2 & 1 \\
\end{array} \right) 
\left( \begin{array}{c}
a \\ b \\ c \\ d \\ e \\ f
\end{array} \right) 
=
\left( \begin{array}{c}
\psi_0 \\ \psi_1 \\ \psi_2 \\ \psi_3 \\ \psi_4 \\ \psi_5
\end{array} \right) 
\end{equation*}
We only need an expression for 
\begin{equation*}
\partial\Psi/\partial x_j = e
\end{equation*}

In [45]:
# Find the finite differene gradient
# Coefficients a,b,c,d,e,f of the polynomial
PolyCoeffs = sy.Matrix(sy.symarray("PolyCoeffs", (6,)))
# Grid point values at j-3,j-2, j-1, j, j+1 and j+2
Psi = sy.Matrix(sy.symarray("Ψ", (6,)), real=True)
Dx = sy.symbols("Δx", real=True)
polyM = Matrix([[-3**5, 3**4, -3**3, 3**2,-3,1],
                [-2**5, 2**4, -2**3, 2**2,-2,1],
                [-1,1,-1,1,-1,1],
                [0,0,0,0,0,1],
                [1,1,1,1,1,1],
                [2**5, 2**4, 2**3, 2**2, 2, 1]])
PolyCoeffs = polyM.solve(Psi)
Ddx = PolyCoeffs[4]/Dx
Ddx
#print(latex(Ddx))

(-Ψ_0/30 + Ψ_1/4 - Ψ_2 + Ψ_3/3 + Ψ_4/2 - Ψ_5/20)/Δx

In [50]:
# Express as a difference of fluxes
# Find Ψ_l and Ψ_r so that Ddx = (Ψ_r - Ψ_l)/dx
DdxR = Ddx*Dx # The residual of Ddx*Dx after removal of (Ψ_r - Ψ_l)
[Psil, Psir] = sy.symbols("Ψ_l, Ψ_r", real=True)
Psil = Psir = sy.S.Zero
for j in range(len(Psi)-1):
    Psil = Psil - Psi[j]*DdxR.coeff(Psi[j])
    Psir = Psir - Psi[j+1]*DdxR.coeff(Psi[j])
    DdxR = sy.collect(sy.expand(Ddx*Dx - (Psir - Psil)), Psi)

print('Residual =', DdxR, '\nThe right flux is')
Psir

Residual = 0 
The right flux is


Ψ_1/30 - 13*Ψ_2/60 + 47*Ψ_3/60 + 9*Ψ_4/20 - Ψ_5/20

In [61]:
# Express $\Psi_r$ as a linear combination of gradients
u = Psi[3]
grad4u = (Psi[1] - 4*Psi[2] + 6*Psi[3] - 4*Psi[4] + Psi[5])/Dx**4
grad2u = (Psi[4] + Psi[2] - 2*Psi[3])/Dx**2
grad2d = (Psi[5] + Psi[3] - 2*Psi[4])/Dx**2

gradGradu = (Psi[1] - Psi[2] - Psi[3] + Psi[4])/(4*Dx**2)
gradd = (Psi[5] - Psi[3])/(2*Dx)
gradu = (Psi[4] - Psi[2])/(2*Dx)
gradf = (Psi[4] - Psi[3])/Dx

#PsirG = u + gradf/2 - grad2u/12 - grad2d/12 + grad4u/30
#PsirG = u + Dx*gradf/2 - Dx*gradd/6 + Dx*gradu/6 + Dx**4*grad4u/30
PsirG = u + Dx*7/30*gradf + 2*Dx*11/60*gradu - 2*Dx*gradd/20 + 4*Dx**2*gradGradu/30
resid = PsirG - Psir
resid

0